In [231]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Essential basic functionality

Here we discuss a lot of the essential functionality common to the pandas data
structures. To begin, let’s create some example objects like we did in
the [10 minutes to pandas](10min.ipynb#min) section:

In [232]:
index = pd.date_range("1/1/2000", periods=8)
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
df = pd.DataFrame(np.random.randn(8, 3), index=index, columns=["A", "B", "C"])


<a id='basics-head-tail'></a>

## Head and tail

To view a small sample of a Series or DataFrame object, use the
`head()` and `tail()` methods. The default number
of elements to display is five, but you may pass a custom number.

In [233]:
long_series = pd.Series(np.random.randn(1000))
long_series.head()
long_series.tail(3)

997   -0.103386
998   -2.341246
999   -2.797939
dtype: float64


<a id='basics-attrs'></a>

## Attributes and underlying data

pandas objects have a number of attributes enabling you to access the metadata

- **shape**: gives the axis dimensions of the object, consistent with ndarray  
- **Axis labels**
- **Series**: *index* (only axis)  
- **DataFrame**: *index* (rows) and *columns*  

Note, **these attributes can be safely assigned to**!

In [234]:
df[:2]
df.columns = [x.lower() for x in df.columns]
df

,a,b,c
2000-01-01,0.109265,-0.391464,-0.091522
2000-01-02,-0.301846,-0.415549,1.223086
2000-01-03,0.198677,-0.158907,0.642420
2000-01-04,0.525761,-1.111169,0.628532
2000-01-05,0.364323,1.241366,1.724102
2000-01-06,-2.222885,0.625753,-1.607452
2000-01-07,0.764395,1.261738,0.170061
2000-01-08,-0.103162,-0.623986,-1.133787


pandas objects (`Index`, `Series`, `DataFrame`) can be
thought of as containers for arrays, which hold the actual data and do the
actual computation. For many types, the underlying array is a
`numpy.ndarray`. However, pandas and 3rd party libraries may *extend*
NumPy’s type system to add support for custom arrays
(see basics.dtypes).

To get the actual data inside a `Index` or `Series`, use
the `.array` property

In [235]:
s.array
s.index.array

<PandasArray>
['a', 'b', 'c', 'd', 'e']
Length: 5, dtype: object

`array` will always be an `ExtensionArray`.
The exact details of what an `ExtensionArray` is and why pandas uses them are a bit
beyond the scope of this introduction. See basics.dtypes for more.

If you know you need a NumPy array, use `to_numpy()`
or `numpy.asarray()`.

In [236]:
s.to_numpy()
np.asarray(s)

array([-0.66533636,  0.91733245,  2.01619275, -0.15712863, -0.31391122])

When the Series or Index is backed by
an `ExtensionArray`, `to_numpy()`
may involve copying data and coercing values. See basics.dtypes for more.

`to_numpy()` gives some control over the `dtype` of the
resulting `numpy.ndarray`. For example, consider datetimes with timezones.
NumPy doesn’t have a dtype to represent timezone-aware datetimes, so there
are two possibly useful representations:

1. An object-dtype `numpy.ndarray` with `Timestamp` objects, each
  with the correct `tz`  
1. A `datetime64[ns]` -dtype `numpy.ndarray`, where the values have
  been converted to UTC and the timezone discarded  


Timezones may be preserved with `dtype=object`

In [237]:
ser = pd.Series(pd.date_range("2000", periods=2, tz="CET"))
ser.to_numpy(dtype=object)

array([Timestamp('2000-01-01 00:00:00+0100', tz='CET', freq='D'),
       Timestamp('2000-01-02 00:00:00+0100', tz='CET', freq='D')],
      dtype=object)

Or thrown away with `dtype='datetime64[ns]'`

In [238]:
ser.to_numpy(dtype="datetime64[ns]")

array(['1999-12-31T23:00:00.000000000', '2000-01-01T23:00:00.000000000'],
      dtype='datetime64[ns]')

Getting the “raw data” inside a `DataFrame` is possibly a bit more
complex. When your `DataFrame` only has a single data type for all the
columns, `DataFrame.to_numpy()` will return the underlying data:

In [239]:
df.to_numpy()

array([[ 0.10926511, -0.39146426, -0.09152222],
       [-0.30184595, -0.41554945,  1.22308622],
       [ 0.19867729, -0.15890652,  0.64241968],
       [ 0.52576091, -1.11116909,  0.62853225],
       [ 0.36432328,  1.2413658 ,  1.72410185],
       [-2.22288514,  0.62575345, -1.60745212],
       [ 0.7643951 ,  1.26173799,  0.17006137],
       [-0.10316242, -0.62398552, -1.1337871 ]])

If a DataFrame contains homogeneously-typed data, the ndarray can
actually be modified in-place, and the changes will be reflected in the data
structure. For heterogeneous data (e.g. some of the DataFrame’s columns are not
all the same dtype), this will not be the case. The values attribute itself,
unlike the axis labels, cannot be assigned to.

>**Note**
>
>When working with heterogeneous data, the dtype of the resulting ndarray
will be chosen to accommodate all of the data involved. For example, if
strings are involved, the result will be of object dtype. If there are only
floats and integers, the resulting array will be of float dtype.

In the past, pandas recommended `Series.values` or `DataFrame.values`
for extracting the data from a Series or DataFrame. You’ll still find references
to these in old code bases and online. Going forward, we recommend avoiding
`.values` and using `.array` or `.to_numpy()`. `.values` has the following
drawbacks:

1. When your Series contains an extension type, it’s
  unclear whether `Series.values` returns a NumPy array or the extension array.
  `Series.array` will always return an `ExtensionArray`, and will never
  copy data. `Series.to_numpy()` will always return a NumPy array,
  potentially at the cost of copying / coercing values.  
1. When your DataFrame contains a mixture of data types, `DataFrame.values` may
  involve copying data and coercing values to a common dtype, a relatively expensive
  operation. `DataFrame.to_numpy()`, being a method, makes it clearer that the
  returned NumPy array may not be a view on the same data in the DataFrame.  



<a id='basics-accelerate'></a>

## Accelerated operations

pandas has support for accelerating certain types of binary numerical and boolean operations using
the `numexpr` library and the `bottleneck` libraries.

These libraries are especially useful when dealing with large data sets, and provide large
speedups. `numexpr` uses smart chunking, caching, and multiple cores. `bottleneck` is
a set of specialized cython routines that are especially fast when dealing with arrays that have
`nans`.

Here is a sample (using 100 column x 100,000 row `DataFrames`):

``````|Operation|0.11.0 (ms)|Prior Version (ms)|Ratio to Prior|
|:-----------------------:|:-----------------------:|:-----------------------:|:-----------------------:|
|df1 > df2|13.32|125.35|0.1063|
|df1 * df2|21.71|36.63|0.5928|
|df1 + df2|22.04|36.50|0.6039|
You are highly encouraged to install both libraries. See the section
[Recommended Dependencies](../01_getting_started/install.ipynb#install-recommended-dependencies) for more installation info.

These are both enabled to be used by default, you can control this by setting the options:

```python
pd.set_option("compute.use_bottleneck", False)
pd.set_option("compute.use_numexpr", False)
```



<a id='basics-binop'></a>

## Flexible binary operations

With binary operations between pandas data structures, there are two key points
of interest:

- Broadcasting behavior between higher- (e.g. DataFrame) and
  lower-dimensional (e.g. Series) objects.  
- Missing data in computations.  


We will demonstrate how to manage these issues independently, though they can
be handled simultaneously.

### Matching / broadcasting behavior

DataFrame has the methods `add()`, `sub()`,
`mul()`, `div()` and related functions
`radd()`, `rsub()`, …
for carrying out binary operations. For broadcasting behavior,
Series input is of primary interest. Using these functions, you can use to
either match on the *index* or *columns* via the **axis** keyword:

In [240]:
df = pd.DataFrame(
    {
        "one": pd.Series(np.random.randn(3), index=["a", "b", "c"]),
        "two": pd.Series(np.random.randn(4), index=["a", "b", "c", "d"]),
        "three": pd.Series(np.random.randn(3), index=["b", "c", "d"]),
    }
)
df

,one,two,three
a,0.075514,1.092052,NaN
b,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
d,NaN,0.295426,-0.720844


In [241]:
row = df.iloc[1]
column = df["two"]

df.sub(row, axis="columns")

,one,two,three
a,1.105911,0.797439,NaN
b,0.000000,0.000000,0.000000
c,0.001729,-0.066560,-0.372081
d,NaN,0.000814,-1.427260


In [242]:
df.sub(row, axis=1)

,one,two,three
a,1.105911,0.797439,NaN
b,0.000000,0.000000,0.000000
c,0.001729,-0.066560,-0.372081
d,NaN,0.000814,-1.427260


In [243]:
df.sub(column, axis="index")

,one,two,three
a,-1.016537,0.0,NaN
b,-1.325008,0.0,0.411804
c,-1.256720,0.0,0.106282
d,NaN,0.0,-1.016270


In [244]:
df.sub(column, axis=0)

,one,two,three
a,-1.016537,0.0,NaN
b,-1.325008,0.0,0.411804
c,-1.256720,0.0,0.106282
d,NaN,0.0,-1.016270


In [245]:
df_orig = df

Furthermore you can align a level of a MultiIndexed DataFrame with a Series.

In [246]:
dfmi = df.copy()
dfmi.index = pd.MultiIndex.from_tuples(
    [(1, "a"), (1, "b"), (1, "c"), (2, "a")], names=["first", "second"]
)
dfmi.sub(column, axis=0, level="second")

one       two     three
first second                              
1     a      -1.016537  0.000000       NaN
      b      -1.325008  0.000000  0.411804
      c      -1.256720  0.000000  0.106282
2     a            NaN -0.796626 -1.812896

Series and Index also support the `divmod()` builtin. This function takes
the floor division and modulo operation at the same time returning a two-tuple
of the same type as the left hand side. For example:

In [247]:
s = pd.Series(np.arange(10))
s

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
dtype: int32

In [248]:
div, rem = divmod(s, 3)
div

0    0
1    0
2    0
3    1
4    1
5    1
6    2
7    2
8    2
9    3
dtype: int32

In [249]:
rem

0    0
1    1
2    2
3    0
4    1
5    2
6    0
7    1
8    2
9    0
dtype: int32

In [250]:
idx = pd.Index(np.arange(10))
idx

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype='int64')

In [251]:
div, rem = divmod(idx, 3)
div

Int64Index([0, 0, 0, 1, 1, 1, 2, 2, 2, 3], dtype='int64')

In [252]:
rem

Int64Index([0, 1, 2, 0, 1, 2, 0, 1, 2, 0], dtype='int64')

We can also do elementwise `divmod()`:

In [253]:
div, rem = divmod(s, [2, 2, 3, 3, 4, 4, 5, 5, 6, 6])
div

0    0
1    0
2    0
3    1
4    1
5    1
6    1
7    1
8    1
9    1
dtype: int32

In [254]:
rem

0    0
1    1
2    2
3    0
4    0
5    1
6    1
7    2
8    2
9    3
dtype: int32

### Missing data / operations with fill values

In Series and DataFrame, the arithmetic functions have the option of inputting
a *fill_value*, namely a value to substitute when at most one of the values at
a location are missing. For example, when adding two DataFrame objects, you may
wish to treat NaN as 0 unless both DataFrames are missing that value, in which
case the result will be NaN (you can later replace NaN with some other value
using `fillna` if you wish).

In [255]:
df2 = df.copy()
df2["three"]["a"] = 1.0

In [256]:
df

,one,two,three
a,0.075514,1.092052,NaN
b,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
d,NaN,0.295426,-0.720844


In [257]:
df2

,one,two,three
a,0.075514,1.092052,1.000000
b,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
d,NaN,0.295426,-0.720844


In [258]:
df + df2

,one,two,three
a,0.151029,2.184103,NaN
b,-2.060793,0.589224,1.412831
c,-2.057335,0.456104,0.668669
d,NaN,0.590851,-1.441688


In [259]:
df.add(df2, fill_value=0)

,one,two,three
a,0.151029,2.184103,1.000000
b,-2.060793,0.589224,1.412831
c,-2.057335,0.456104,0.668669
d,NaN,0.590851,-1.441688



<a id='basics-compare'></a>

### Flexible comparisons

Series and DataFrame have the binary comparison methods `eq`, `ne`, `lt`, `gt`,
`le`, and `ge` whose behavior is analogous to the binary
arithmetic operations described above:

In [260]:
df.gt(df2)

,one,two,three
a,False,False,False
b,False,False,False
c,False,False,False
d,False,False,False


In [261]:
df2.ne(df)

,one,two,three
a,False,False,True
b,False,False,False
c,False,False,False
d,True,False,False


These operations produce a pandas object of the same type as the left-hand-side
input that is of dtype `bool`. These `boolean` objects can be used in
indexing operations, see the section on Boolean indexing.


<a id='basics-reductions'></a>

### Boolean reductions

You can apply the reductions: `empty`, `any()`,
`all()`, and `bool()` to provide a
way to summarize a boolean result.

In [262]:
(df > 0).all()

one      False
two       True
three    False
dtype: bool

In [263]:
(df > 0).any()

one      True
two      True
three    True
dtype: bool

You can reduce to a final boolean value.

In [264]:
(df > 0).any().any()

True

You can test if a pandas object is empty, via the `empty` property.

In [265]:
df.empty

False

In [266]:
pd.DataFrame(columns=list("ABC")).empty

True

To evaluate single-element pandas objects in a boolean context, use the method
`bool()`:

In [267]:
pd.Series([True]).bool()

True

In [268]:
pd.Series([False]).bool()

False

In [269]:
pd.DataFrame([[True]]).bool()
pd.DataFrame([[False]]).bool()

False

You might be tempted to do the following:

> if df:<BR>
> &nbsp;&nbsp;&nbsp;&nbsp;pass
>
> or
>
> df and df2    


These will both raise errors, as you are trying to compare multiple values.::

    ValueError: The truth value of an array is ambiguous. Use a.empty, a.any() or a.all().

See [gotchas](43_FAQ_gotchas.ipynb#gotchas-truth) for a more detailed discussion.


<a id='basics-equals'></a>

### Comparing if objects are equivalent

Often you may find that there is more than one way to compute the same
result.  As a simple example, consider `df + df` and `df * 2`. To test
that these two computations produce the same result, given the tools
shown above, you might imagine using `(df + df == df * 2).all()`. But in
fact, this expression is False:

In [270]:
df + df == df * 2
(df + df == df * 2).all()

one      False
two       True
three    False
dtype: bool

Notice that the boolean DataFrame `df + df == df * 2` contains some False values!
This is because NaNs do not compare as equals:

In [271]:
np.nan == np.nan

False

So, NDFrames (such as Series and DataFrames)
have an `equals()` method for testing equality, with NaNs in
corresponding locations treated as equal.

In [272]:
(df + df).equals(df * 2)

True

Note that the Series or DataFrame index needs to be in the same order for
equality to be True:

In [273]:
df1 = pd.DataFrame({"col": ["foo", 0, np.nan]})
df2 = pd.DataFrame({"col": [np.nan, 0, "foo"]}, index=[2, 1, 0])
df1.equals(df2)


False

In [274]:
df1.equals(df2.sort_index())

True

### Comparing array-like objects

You can conveniently perform element-wise comparisons when comparing a pandas
data structure with a scalar value:

In [275]:
pd.Series(["foo", "bar", "baz"]) == "foo"
pd.Index(["foo", "bar", "baz"]) == "foo"

array([ True, False, False])

pandas also handles element-wise comparisons between different array-like
objects of the same length:

In [276]:
pd.Series(["foo", "bar", "baz"]) == pd.Index(["foo", "bar", "qux"])
pd.Series(["foo", "bar", "baz"]) == np.array(["foo", "bar", "qux"])

0     True
1     True
2    False
dtype: bool

Trying to compare `Index` or `Series` objects of different lengths will
raise a ValueError:

```python
In [55]: pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo', 'bar'])
ValueError: Series lengths must match to compare

In [56]: pd.Series(['foo', 'bar', 'baz']) == pd.Series(['foo'])
ValueError: Series lengths must match to compare
```


Note that this is different from the NumPy behavior where a comparison can
be broadcast:

In [277]:
np.array([1, 2, 3]) == np.array([2])

array([False,  True, False])

or it can return False if broadcasting can not be done:

In [278]:
np.array([1, 2, 3]) == np.array([1, 2])
#<ipython-input-58-7a76bc07877c>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.

<ipython-input-278-8b3fe6b3be63>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  np.array([1, 2, 3]) == np.array([1, 2])


False

### Combining overlapping data sets

A problem occasionally arising is the combination of two similar data sets
where values in one are preferred over the other. An example would be two data
series representing a particular economic indicator where one is considered to
be of “higher quality”. However, the lower quality series might extend further
back in history or have more complete data coverage. As such, we would like to
combine two DataFrame objects where missing values in one DataFrame are
conditionally filled with like-labeled values from the other DataFrame. The
function implementing this operation is `combine_first()`,
which we illustrate:

In [279]:
df1 = pd.DataFrame(
    {"A": [1.0, np.nan, 3.0, 5.0, np.nan], "B": [np.nan, 2.0, 3.0, np.nan, 6.0]}
)
df2 = pd.DataFrame(
    {
        "A": [5.0, 2.0, 4.0, np.nan, 3.0, 7.0],
        "B": [np.nan, np.nan, 3.0, 4.0, 6.0, 8.0],
    }
)
df1

,A,B
0,1.0,NaN
1,NaN,2.0
2,3.0,3.0
3,5.0,NaN
4,NaN,6.0


In [280]:
df2

,A,B
0,5.0,NaN
1,2.0,NaN
2,4.0,3.0
3,NaN,4.0
4,3.0,6.0
5,7.0,8.0


In [281]:
df1.combine_first(df2)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0


### General DataFrame combine

The `combine_first()` method above calls the more general
`DataFrame.combine()`. This method takes another DataFrame
and a combiner function, aligns the input DataFrame and then passes the combiner
function pairs of Series (i.e., columns whose names are the same).

So, for instance, to reproduce `combine_first()` as above:

In [282]:
def combiner(x, y):
    return np.where(pd.isna(x), y, x)


df1.combine(df2, combiner)

,A,B
0,1.0,NaN
1,2.0,2.0
2,3.0,3.0
3,5.0,4.0
4,3.0,6.0
5,7.0,8.0



<a id='basics-stats'></a>

## Descriptive statistics

There exists a large number of methods for computing descriptive statistics and
other related operations on Series, [DataFrame](../05_reference/13_dataFrame.ipynb#api-dataframe-stats). Most of these
are aggregations (hence producing a lower-dimensional result) like
`sum()`, `mean()`, and `quantile()`,
but some of them, like `cumsum()` and `cumprod()`,
produce an object of the same size. Generally speaking, these methods take an
**axis** argument, just like *ndarray.{sum, std, …}*, but the axis can be
specified by name or integer:

- **Series**: no axis argument needed  
- **DataFrame**: “index” (axis=0, default), “columns” (axis=1)  


For example:

In [283]:
df

,one,two,three
a,0.075514,1.092052,NaN
b,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
d,NaN,0.295426,-0.720844


In [284]:
df.mean(0)

one     -0.661183
two      0.477535
three    0.106635
dtype: float64

In [285]:
df.mean(1)

a    0.583783
b   -0.009789
c   -0.155427
d   -0.212709
dtype: float64

All such methods have a `skipna` option signaling whether to exclude missing
data (`True` by default):

In [286]:
df.sum(0, skipna=False)

one           NaN
two      1.910142
three         NaN
dtype: float64

In [287]:
df.sum(axis=1, skipna=True)

a    1.167566
b   -0.029368
c   -0.466281
d   -0.425418
dtype: float64

Combined with the broadcasting / arithmetic behavior, one can describe various
statistical procedures, like standardization (rendering data zero mean and
standard deviation of 1), very concisely:

In [288]:
ts_stand = (df - df.mean()) / df.std()
ts_stand.std()

one      1.0
two      1.0
three    1.0
dtype: float64

In [289]:
xs_stand = df.sub(df.mean(1), axis=0).div(df.std(1), axis=0)
xs_stand.std(1)

a    1.0
b    1.0
c    1.0
d    1.0
dtype: float64

Note that methods like `cumsum()` and `cumprod()`
preserve the location of `NaN` values. This is somewhat different from
`expanding()` and `rolling()` since `NaN` behavior
is furthermore dictated by a `min_periods` parameter.

In [290]:
df.cumsum()

,one,two,three
a,0.075514,1.092052,NaN
b,-0.954882,1.386664,0.706416
c,-1.983549,1.614716,1.040750
d,NaN,1.910142,0.319906


Here is a quick reference summary table of common functions. Each also takes an
optional `level` parameter which applies only if the object has a
hierarchical index.

|Function|Description|
|:------------------|:------------------------------------------------------------------------------|
|count|Number of non-NA observations|
|sum|Sum of values|
|mean|Mean of values|
|mad|Mean absolute deviation|
|median|Arithmetic median of values|
|min|Minimum|
|max|Maximum|
|mode|Mode|
|abs|Absolute Value|
|prod|Product of values|
|std|Bessel-corrected sample standard deviation|
|var|Unbiased variance|
|sem|Standard error of the mean|
|skew|Sample skewness (3rd moment)|
|kurt|Sample kurtosis (4th moment)|
|quantile|Sample quantile (value at %)|
|cumsum|Cumulative sum|
|cumprod|Cumulative product|
|cummax|Cumulative maximum|
|cummin|Cumulative minimum|


Note that by chance some NumPy methods, like `mean`, `std`, and `sum`,
will exclude NAs on Series input by default:

In [291]:
np.mean(df["one"])

-0.6611831366807629

In [292]:
np.mean(df["one"].to_numpy())

nan

`Series.nunique()` will return the number of unique non-NA values in a
Series:

In [293]:
series = pd.Series(np.random.randn(500))
series[20:500] = np.nan
series[10:20] = 5
series.nunique()

11


<a id='basics-describe'></a>

### Summarizing data: describe

There is a convenient `describe()` function which computes a variety of summary
statistics about a Series or the columns of a DataFrame (excluding NAs of
course):

In [294]:
series = pd.Series(np.random.randn(1000))
series[::2] = np.nan
series.describe()

count    500.000000
mean       0.127847
std        1.044871
min       -3.474301
25%       -0.563509
50%        0.160509
75%        0.838872
max        3.233916
dtype: float64

In [295]:
frame = pd.DataFrame(np.random.randn(1000, 5), columns=["a", "b", "c", "d", "e"])
frame.iloc[::2] = np.nan
frame.describe()

,a,b,c,d,e
count,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.100002,-0.034309,-0.005170,0.000448,-0.026713
std,0.931921,0.954818,0.961490,0.989543,1.045745
min,-2.472430,-3.281907,-3.093750,-3.373955,-2.870281
25%,-0.561391,-0.664603,-0.605527,-0.677428,-0.711042
50%,0.120135,-0.117833,-0.005011,0.032045,-0.031044
75%,0.678720,0.594165,0.609173,0.650596,0.685870
max,3.481073,2.741764,2.727634,3.328725,3.777564


You can select specific percentiles to include in the output:

In [296]:
series.describe(percentiles=[0.05, 0.25, 0.75, 0.95])

count    500.000000
mean       0.127847
std        1.044871
min       -3.474301
5%        -1.623751
25%       -0.563509
50%        0.160509
75%        0.838872
95%        1.887923
max        3.233916
dtype: float64

By default, the median is always included.

For a non-numerical Series object, `describe()` will give a simple
summary of the number of unique values and most frequently occurring values:

In [297]:
s = pd.Series(["a", "a", "b", "b", "a", "a", np.nan, "c", "d", "a"])
s.describe()

count     9
unique    4
top       a
freq      5
dtype: object

Note that on a mixed-type DataFrame object, `describe()` will
restrict the summary to include only numerical columns or, if none are, only
categorical columns:

In [298]:
frame = pd.DataFrame({"a": ["Yes", "Yes", "No", "No"], "b": range(4)})
frame.describe()

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


This behavior can be controlled by providing a list of types as `include`/`exclude`
arguments. The special value `all` can also be used:

In [299]:
frame.describe(include=["object"])

,a
count,4
unique,2
top,Yes
freq,2


In [300]:
frame.describe(include=["number"])

,b
count,4.000000
mean,1.500000
std,1.290994
min,0.000000
25%,0.750000
50%,1.500000
75%,2.250000
max,3.000000


In [301]:
frame.describe(include="all")

,a,b
count,4,4.000000
unique,2,NaN
top,Yes,NaN
freq,2,NaN
mean,NaN,1.500000
std,NaN,1.290994
min,NaN,0.000000
25%,NaN,0.750000
50%,NaN,1.500000
75%,NaN,2.250000


That feature relies on select_dtypes. Refer to
there for details about accepted inputs.


<a id='basics-idxmin'></a>

### Index of min/max values

The `idxmin()` and `idxmax()` functions on Series
and DataFrame compute the index labels with the minimum and maximum
corresponding values:

In [302]:
s1 = pd.Series(np.random.randn(5))
s1

0    1.640724
1    0.298740
2    0.159868
3   -0.261832
4    1.322617
dtype: float64

In [303]:
s1.idxmin(), s1.idxmax()

(3, 0)

In [304]:
df1 = pd.DataFrame(np.random.randn(5, 3), columns=["A", "B", "C"])
df1

,A,B,C
0,-0.379387,-1.241765,0.727141
1,-0.181962,1.971804,-0.054458
2,-0.568901,1.572170,-0.090328
3,-0.723367,0.681294,1.065787
4,-0.497899,0.572091,-3.430577


In [305]:
df1.idxmin(axis=0)

A    3
B    0
C    4
dtype: int64

In [306]:
df1.idxmax(axis=1)

0    C
1    B
2    B
3    C
4    B
dtype: object

When there are multiple rows (or columns) matching the minimum or maximum
value, `idxmin()` and `idxmax()` return the first
matching index:

In [307]:
df3 = pd.DataFrame([2, 1, 1, 3, np.nan], columns=["A"], index=list("edcba"))
df3

,A
e,2.0
d,1.0
c,1.0
b,3.0
a,NaN


In [308]:
df3["A"].idxmin()

'd'

>**Note**
>
>`idxmin` and `idxmax` are called `argmin` and `argmax` in NumPy.


<a id='basics-discretization'></a>

### Value counts (histogramming) / mode

The `value_counts()` Series method and top-level function computes a histogram
of a 1D array of values. It can also be used as a function on regular arrays:

In [309]:
data = np.random.randint(0, 7, size=50)
data

array([2, 5, 6, 6, 3, 4, 3, 2, 4, 5, 0, 6, 3, 0, 1, 5, 4, 2, 0, 6, 3, 1,
       4, 5, 6, 4, 6, 6, 5, 4, 2, 0, 4, 5, 4, 5, 0, 1, 5, 0, 0, 0, 2, 5,
       0, 1, 0, 6, 2, 3])

In [310]:
s = pd.Series(data)
s.value_counts()

0    10
5     9
4     8
6     8
2     6
3     5
1     4
dtype: int64

In [311]:
pd.value_counts(data)

0    10
5     9
4     8
6     8
2     6
3     5
1     4
dtype: int64

New in version 1.1.0.

The `value_counts()` method can be used to count combinations across multiple columns.
By default all columns are used but a subset can be selected using the `subset` argument.

In [312]:
data = {"a": [1, 2, 3, 4], "b": ["x", "x", "y", "y"]}
frame = pd.DataFrame(data)
frame.value_counts()

a  b
1  x    1
2  x    1
3  y    1
4  y    1
dtype: int64

Similarly, you can get the most frequently occurring value(s), i.e. the mode, of the values in a Series or DataFrame:

In [313]:
s5 = pd.Series([1, 1, 3, 3, 3, 5, 5, 7, 7, 7])
s5.mode()
df5 = pd.DataFrame(
    {
        "A": np.random.randint(0, 7, size=50),
        "B": np.random.randint(-10, 15, size=50),
    }
)
df5.mode()

,A,B
0,3.0,1
1,NaN,10


### Discretization and quantiling

Continuous values can be discretized using the `cut()` (bins based on values)
and `qcut()` (bins based on sample quantiles) functions:

In [314]:
arr = np.random.randn(20)
factor = pd.cut(arr, 4)
factor

[(-1.16, -0.361], (-1.16, -0.361], (0.434, 1.23], (-1.16, -0.361], (1.23, 2.025], ..., (-0.361, 0.434], (0.434, 1.23], (1.23, 2.025], (-1.16, -0.361], (-0.361, 0.434]]
Length: 20
Categories (4, interval[float64]): [(-1.16, -0.361] < (-0.361, 0.434] < (0.434, 1.23] < (1.23, 2.025]]

In [315]:
factor = pd.cut(arr, [-5, -1, 0, 1, 5])
factor

[(-1, 0], (-1, 0], (1, 5], (-1, 0], (1, 5], ..., (-1, 0], (1, 5], (1, 5], (-1, 0], (-1, 0]]
Length: 20
Categories (4, interval[int64]): [(-5, -1] < (-1, 0] < (0, 1] < (1, 5]]

`qcut()` computes sample quantiles. For example, we could slice up some
normally distributed data into equal-size quartiles like so:

In [316]:
arr = np.random.randn(30)
factor = pd.qcut(arr, [0, 0.25, 0.5, 0.75, 1])
factor

[(-2.116, -0.417], (-2.116, -0.417], (0.935, 1.599], (0.935, 1.599], (0.216, 0.935], ..., (-0.417, 0.216], (-0.417, 0.216], (-2.116, -0.417], (0.216, 0.935], (0.935, 1.599]]
Length: 30
Categories (4, interval[float64]): [(-2.116, -0.417] < (-0.417, 0.216] < (0.216, 0.935] < (0.935, 1.599]]

In [317]:
pd.value_counts(factor)

(-2.116, -0.417]    8
(0.935, 1.599]      8
(-0.417, 0.216]     7
(0.216, 0.935]      7
dtype: int64

We can also pass infinite values to define the bins:

In [318]:
arr = np.random.randn(20)
factor = pd.cut(arr, [-np.inf, 0, np.inf])
factor

[(-inf, 0.0], (-inf, 0.0], (-inf, 0.0], (0.0, inf], (0.0, inf], ..., (-inf, 0.0], (-inf, 0.0], (0.0, inf], (-inf, 0.0], (0.0, inf]]
Length: 20
Categories (2, interval[float64]): [(-inf, 0.0] < (0.0, inf]]


<a id='basics-apply'></a>

## Function application

To apply your own or another library’s functions to pandas objects,
you should be aware of the three methods below. The appropriate
method to use depends on whether your function expects to operate
on an entire `DataFrame` or `Series`, row- or column-wise, or elementwise.

1. [Tablewise Function Application](#tablewise-function-application): `pipe()`  
1. [Row or Column-wise Function Application](#row-or-column-wise-function-application): `apply()`  
1. [Aggregation API](#aggregation-api): `agg()` and `transform()`  
1. [Applying Elementwise Functions](#applying-elementwise-functions): `applymap()`  



<a id='basics-pipe'></a>

### Tablewise function application

`DataFrames` and `Series` can be passed into functions.
However, if the function needs to be called in a chain, consider using the `pipe()` method.

First some setup:

In [319]:
def extract_city_name(df):
    """
    Chicago, IL -> Chicago for city_name column
    """
    df["city_name"] = df["city_and_code"].str.split(",").str.get(0)
    return df


def add_country_name(df, country_name=None):
    """
    Chicago -> Chicago-US for city_name column
    """
    col = "city_name"
    df["city_and_country"] = df[col] + country_name
    return df


df_p = pd.DataFrame({"city_and_code": ["Chicago, IL"]})
df_p

,city_and_code
0,"Chicago, IL"


`extract_city_name` and `add_country_name` are functions taking and returning `DataFrames`.

Now compare the following:

In [320]:
add_country_name(extract_city_name(df_p), country_name="US")

,city_and_code,city_name,city_and_country
0,"Chicago, IL",Chicago,ChicagoUS


Is equivalent to:

In [321]:
df_p.pipe(extract_city_name).pipe(add_country_name, country_name="US")

,city_and_code,city_name,city_and_country
0,"Chicago, IL",Chicago,ChicagoUS


pandas encourages the second style, which is known as method chaining.
`pipe` makes it easy to use your own or another library’s functions
in method chains, alongside pandas’ methods.

In the example above, the functions `extract_city_name` and `add_country_name` each expected a `DataFrame` as the first positional argument.
What if the function you wish to apply takes its data as, say, the second argument?
In this case, provide `pipe` with a tuple of `(callable, data_keyword)`.
`.pipe` will route the `DataFrame` to the argument specified in the tuple.

For example, we can fit a regression using statsmodels. Their API expects a formula first and a `DataFrame` as the second argument, `data`. We pass in the function, keyword pair `(sm.ols, 'data')` to `pipe`:

In [322]:
import statsmodels.formula.api as sm

bb = pd.read_csv("../data/baseball.csv", index_col="id")

(
    bb.query("h > 0")
    .assign(ln_h=lambda df: np.log(df.h))
    .pipe((sm.ols, "data"), "hr ~ ln_h + year + g + C(lg)")
    .fit()
    .summary()
)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     hr   R-squared:                       0.685
Model:                            OLS   Adj. R-squared:                  0.665
Method:                 Least Squares   F-statistic:                     34.28
Date:                Mon, 30 Aug 2021   Prob (F-statistic):           3.48e-15
Time:                        09:11:34   Log-Likelihood:                -205.92
No. Observations:                  68   AIC:                             421.8
Df Residuals:                      63   BIC:                             432.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -8484.7720   4664.146     -1.819      0.074   -1.78e+04     835.780
C(lg)[T.NL]    -2.2736      1.325     -1.716      0.091      -4.922       0.375
ln_h           -1.3542      0.875     -1.547      0.127      -3.103       0.395
year            4.2277      2.324      1.819      0.074      -0.417       8.872
g               0.1841      0.029      6.258      0.000       0.125       0.243
==============================================================================
Omnibus:                       10.875   Durbin-Watson:                   1.999
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               17.298
Skew:                           0.537   Prob(JB):                     0.000175
Kurtosis:                       5.225   Cond. No.                     1.49e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.49e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The pipe method is inspired by unix pipes and more recently [dplyr](https://github.com/hadley/dplyr) and [magrittr](https://github.com/smbache/magrittr), which
have introduced the popular `(%>%)` (read pipe) operator for [R](https://www.r-project.org).
The implementation of `pipe` here is quite clean and feels right at home in Python.
We encourage you to view the source code of `pipe()`.

### Row or column-wise function application

Arbitrary functions can be applied along the axes of a DataFrame
using the `apply()` method, which, like the descriptive
statistics methods, takes an optional `axis` argument:

In [323]:
df.apply(np.mean)

one     -0.661183
two      0.477535
three    0.106635
dtype: float64

In [324]:
df.apply(np.mean, axis=1)

a    0.583783
b   -0.009789
c   -0.155427
d   -0.212709
dtype: float64

In [325]:
df.apply(lambda x: x.max() - x.min())

one      1.105911
two      0.863999
three    1.427260
dtype: float64

In [326]:
df.apply(np.cumsum)

,one,two,three
a,0.075514,1.092052,NaN
b,-0.954882,1.386664,0.706416
c,-1.983549,1.614716,1.040750
d,NaN,1.910142,0.319906


In [327]:
df.apply(np.exp)

,one,two,three
a,1.078439,2.980382,NaN
b,0.356866,1.342606,2.026714
c,0.357483,1.256151,1.397010
d,NaN,1.343698,0.486342


The `apply()` method will also dispatch on a string method name.

In [328]:
df.apply("mean")

one     -0.661183
two      0.477535
three    0.106635
dtype: float64

In [329]:
df.apply("mean", axis=1)

a    0.583783
b   -0.009789
c   -0.155427
d   -0.212709
dtype: float64

The return type of the function passed to `apply()` affects the
type of the final output from `DataFrame.apply` for the default behaviour:

- If the applied function returns a `Series`, the final output is a `DataFrame`.
  The columns match the index of the `Series` returned by the applied function.  
- If the applied function returns any other type, the final output is a `Series`.  


This default behaviour can be overridden using the `result_type`, which
accepts three options: `reduce`, `broadcast`, and `expand`.
These will determine how list-likes return values expand (or not) to a `DataFrame`.

`apply()` combined with some cleverness can be used to answer many questions
about a data set. For example, suppose we wanted to extract the date where the
maximum value for each column occurred:

In [330]:
tsdf = pd.DataFrame(
    np.random.randn(1000, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=1000),
)
tsdf.apply(lambda x: x.idxmax())

A   2000-10-12
B   2001-05-18
C   2000-12-30
dtype: datetime64[ns]

You may also pass additional arguments and keyword arguments to the `apply()`
method. For instance, consider the following function you would like to apply:

In [331]:
def subtract_and_divide(x, sub, divide=1):
    return (x - sub) / divide


You may then apply this function as follows:

In [332]:
df.apply(subtract_and_divide, args=(5,), divide=3)
# TypeError: unsupported operand type(s) for -: 'str' and 'int'

,one,two,three
a,-1.641495,-1.302649,NaN
b,-2.010132,-1.568463,-1.431195
c,-2.009556,-1.590649,-1.555222
d,NaN,-1.568191,-1.906948


Another useful feature is the ability to pass Series methods to carry out some
Series operation on each column or row:

In [333]:
tsdf = pd.DataFrame(
    np.random.randn(10, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=10),
)
tsdf.iloc[3:7] = np.nan
tsdf

,A,B,C
2000-01-01,0.291460,0.589208,-0.294584
2000-01-02,-0.308003,-1.898961,-0.407429
2000-01-03,-0.442057,-2.553411,-1.819661
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.816031,-0.113389,0.269551
2000-01-09,-1.973795,-1.015897,1.558662
2000-01-10,0.455181,-1.901443,-1.422891


In [334]:
tsdf.apply(pd.Series.interpolate)

,A,B,C
2000-01-01,0.291460,0.589208,-0.294584
2000-01-02,-0.308003,-1.898961,-0.407429
2000-01-03,-0.442057,-2.553411,-1.819661
2000-01-04,-0.190439,-2.065406,-1.401818
2000-01-05,0.061178,-1.577402,-0.983976
2000-01-06,0.312796,-1.089397,-0.566134
2000-01-07,0.564414,-0.601393,-0.148291
2000-01-08,0.816031,-0.113389,0.269551
2000-01-09,-1.973795,-1.015897,1.558662
2000-01-10,0.455181,-1.901443,-1.422891


Finally, `apply()` takes an argument `raw` which is False by default, which
converts each row or column into a Series before applying the function. When
set to True, the passed function will instead receive an ndarray object, which
has positive performance implications if you do not need the indexing
functionality.


<a id='basics-aggregate'></a>

### Aggregation API

The aggregation API allows one to express possibly multiple aggregation operations in a single concise way.
This API is similar across pandas objects, see [groupby API](36_groupby.ipynb#groupby-aggregate), the
[window API](37_window.ipynb#window-overview), and the [resample API](38_timeseries.ipynb#timeseries-aggregate).
The entry point for aggregation is `DataFrame.aggregate()`, or the alias
`DataFrame.agg()`.

We will use a similar starting frame from above:

In [335]:
tsdf = pd.DataFrame(
    np.random.randn(10, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=10),
)
tsdf.iloc[3:7] = np.nan
tsdf

,A,B,C
2000-01-01,-0.059854,-1.406932,0.598150
2000-01-02,1.120896,-0.784340,0.209990
2000-01-03,-0.364974,0.081641,-0.635384
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.131367,0.164194,-0.613220
2000-01-09,-1.152547,1.513897,-1.110896
2000-01-10,-1.837484,-0.358004,1.348824


Using a single function is equivalent to `apply()`. You can also
pass named methods as strings. These will return a `Series` of the aggregated
output:

In [336]:
tsdf.agg(np.sum)

tsdf.agg("sum")

# these are equivalent to a ``.sum()`` because we are aggregating
# on a single function
tsdf.sum()

A   -2.162596
B   -0.789545
C   -0.202536
dtype: float64

Single aggregations on a `Series` this will return a scalar value:

In [337]:
tsdf["A"].agg("sum")

-2.16259565329887

#### Aggregating with multiple functions

You can pass multiple aggregation arguments as a list.
The results of each of the passed functions will be a row in the resulting `DataFrame`.
These are naturally named from the aggregation function.

In [338]:
tsdf.agg(["sum"])

,A,B,C
sum,-2.162596,-0.789545,-0.202536


Multiple functions yield multiple rows:

In [339]:
tsdf.agg(["sum", "mean"])

,A,B,C
sum,-2.162596,-0.789545,-0.202536
mean,-0.360433,-0.131591,-0.033756


On a `Series`, multiple functions return a `Series`, indexed by the function names:

In [340]:
tsdf["A"].agg(["sum", "mean"])

sum    -2.162596
mean   -0.360433
Name: A, dtype: float64

Passing a `lambda` function will yield a `<lambda>` named row:

In [341]:
tsdf["A"].agg(["sum", lambda x: x.mean()])

sum        -2.162596
<lambda>   -0.360433
Name: A, dtype: float64

Passing a named function will yield that name for the row:

In [342]:
def mymean(x):
    return x.mean()


tsdf["A"].agg(["sum", mymean])

sum      -2.162596
mymean   -0.360433
Name: A, dtype: float64

#### Aggregating with a dict

Passing a dictionary of column names to a scalar or a list of scalars, to `DataFrame.agg`
allows you to customize which functions are applied to which columns. Note that the results
are not in any particular order, you can use an `OrderedDict` instead to guarantee ordering.

In [343]:
tsdf.agg({"A": "mean", "B": "sum"})

A   -0.360433
B   -0.789545
dtype: float64

Passing a list-like will generate a `DataFrame` output. You will get a matrix-like output
of all of the aggregators. The output will consist of all unique functions. Those that are
not noted for a particular column will be `NaN`:

In [344]:
tsdf.agg({"A": ["mean", "min"], "B": "sum"})

,A,B
mean,-0.360433,NaN
min,-1.837484,NaN
sum,NaN,-0.789545



<a id='basics-aggregation-mixed-string'></a>

#### Mixed dtypes

When presented with mixed dtypes that cannot aggregate, `.agg` will only take the valid
aggregations. This is similar to how `.groupby.agg` works.

In [345]:
mdf = pd.DataFrame(
    {
        "A": [1, 2, 3],
        "B": [1.0, 2.0, 3.0],
        "C": ["foo", "bar", "baz"],
        "D": pd.date_range("20130101", periods=3),
    }
)
mdf.dtypes

A             int64
B           float64
C            object
D    datetime64[ns]
dtype: object

In [346]:
mdf.agg(["min", "sum"])

,A,B,C,D
min,1,1.0,bar,2013-01-01
sum,6,6.0,foobarbaz,NaT



<a id='basics-aggregation-custom-describe'></a>

#### Custom describe

With `.agg()` it is possible to easily create a custom describe function, similar
to the built in describe function.

In [347]:
from functools import partial

q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = "25%"
q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = "75%"

tsdf.agg(["count", "mean", "std", "min", q_25, "median", q_75, "max"])

,A,B,C
count,6.000000,6.000000,6.000000
mean,-0.360433,-0.131591,-0.033756
std,1.033024,0.994427,0.919588
min,-1.837484,-1.406932,-1.110896
25%,-0.955654,-0.677756,-0.629843
median,-0.212414,-0.138182,-0.201615
75%,0.083562,0.143556,0.501110
max,1.120896,1.513897,1.348824



<a id='basics-transform'></a>

### Transform API

The `transform()` method returns an object that is indexed the same (same size)
as the original. This API allows you to provide *multiple* operations at the same
time rather than one-by-one. Its API is quite similar to the `.agg` API.

We create a frame similar to the one used in the above sections.

In [348]:
tsdf = pd.DataFrame(
    np.random.randn(10, 3),
    columns=["A", "B", "C"],
    index=pd.date_range("1/1/2000", periods=10),
)
tsdf.iloc[3:7] = np.nan
tsdf

,A,B,C
2000-01-01,-1.026971,-1.602870,0.071241
2000-01-02,0.758735,0.861963,0.675649
2000-01-03,1.699417,0.808933,1.193305
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.883390,0.335357,0.992876
2000-01-09,-0.498776,-0.684497,0.870068
2000-01-10,-1.232448,1.043798,0.034777


Transform the entire frame. `.transform()` allows input functions as: a NumPy function, a string
function name or a user defined function.

In [349]:
tsdf.transform(np.abs)

,A,B,C
2000-01-01,1.026971,1.602870,0.071241
2000-01-02,0.758735,0.861963,0.675649
2000-01-03,1.699417,0.808933,1.193305
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.883390,0.335357,0.992876
2000-01-09,0.498776,0.684497,0.870068
2000-01-10,1.232448,1.043798,0.034777


In [350]:
tsdf.transform("abs")

,A,B,C
2000-01-01,1.026971,1.602870,0.071241
2000-01-02,0.758735,0.861963,0.675649
2000-01-03,1.699417,0.808933,1.193305
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.883390,0.335357,0.992876
2000-01-09,0.498776,0.684497,0.870068
2000-01-10,1.232448,1.043798,0.034777


In [351]:
tsdf.transform(lambda x: x.abs())

,A,B,C
2000-01-01,1.026971,1.602870,0.071241
2000-01-02,0.758735,0.861963,0.675649
2000-01-03,1.699417,0.808933,1.193305
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.883390,0.335357,0.992876
2000-01-09,0.498776,0.684497,0.870068
2000-01-10,1.232448,1.043798,0.034777


Here `transform()` received a single function; this is equivalent to a [ufunc](https://numpy.org/doc/stable/reference/ufuncs.html) application.

In [352]:
np.abs(tsdf)

,A,B,C
2000-01-01,1.026971,1.602870,0.071241
2000-01-02,0.758735,0.861963,0.675649
2000-01-03,1.699417,0.808933,1.193305
2000-01-04,NaN,NaN,NaN
2000-01-05,NaN,NaN,NaN
2000-01-06,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN
2000-01-08,0.883390,0.335357,0.992876
2000-01-09,0.498776,0.684497,0.870068
2000-01-10,1.232448,1.043798,0.034777


Passing a single function to `.transform()` with a `Series` will yield a single `Series` in return.

In [353]:
tsdf["A"].transform(np.abs)

2000-01-01    1.026971
2000-01-02    0.758735
2000-01-03    1.699417
2000-01-04         NaN
2000-01-05         NaN
2000-01-06         NaN
2000-01-07         NaN
2000-01-08    0.883390
2000-01-09    0.498776
2000-01-10    1.232448
Freq: D, Name: A, dtype: float64

#### Transform with multiple functions

Passing multiple functions will yield a column MultiIndexed DataFrame.
The first level will be the original frame column names; the second level
will be the names of the transforming functions.

In [354]:
tsdf.transform([np.abs, lambda x: x + 1])

A                   B                   C          
            absolute  <lambda>  absolute  <lambda>  absolute  <lambda>
2000-01-01  1.026971 -0.026971  1.602870 -0.602870  0.071241  1.071241
2000-01-02  0.758735  1.758735  0.861963  1.861963  0.675649  1.675649
2000-01-03  1.699417  2.699417  0.808933  1.808933  1.193305  2.193305
2000-01-04       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN       NaN       NaN       NaN
2000-01-08  0.883390  1.883390  0.335357  1.335357  0.992876  1.992876
2000-01-09  0.498776  0.501224  0.684497  0.315503  0.870068  1.870068
2000-01-10  1.232448 -0.232448  1.043798  2.043798  0.034777  1.034777

Passing multiple functions to a Series will yield a DataFrame. The
resulting column names will be the transforming functions.

In [355]:
tsdf["A"].transform([np.abs, lambda x: x + 1])

,absolute,<lambda>
2000-01-01,1.026971,-0.026971
2000-01-02,0.758735,1.758735
2000-01-03,1.699417,2.699417
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,0.883390,1.883390
2000-01-09,0.498776,0.501224
2000-01-10,1.232448,-0.232448


#### Transforming with a dict

Passing a dict of functions will allow selective transforming per column.

In [356]:
tsdf.transform({"A": np.abs, "B": lambda x: x + 1})

,A,B
2000-01-01,1.026971,-0.602870
2000-01-02,0.758735,1.861963
2000-01-03,1.699417,1.808933
2000-01-04,NaN,NaN
2000-01-05,NaN,NaN
2000-01-06,NaN,NaN
2000-01-07,NaN,NaN
2000-01-08,0.883390,1.335357
2000-01-09,0.498776,0.315503
2000-01-10,1.232448,2.043798


Passing a dict of lists will generate a MultiIndexed DataFrame with these
selective transforms.

In [357]:
tsdf.transform({"A": np.abs, "B": [lambda x: x + 1, "sqrt"]})

C:\Users\olippuner\anaconda3\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


A         B          
            absolute  <lambda>      sqrt
2000-01-01  1.026971 -0.602870       NaN
2000-01-02  0.758735  1.861963  0.928419
2000-01-03  1.699417  1.808933  0.899407
2000-01-04       NaN       NaN       NaN
2000-01-05       NaN       NaN       NaN
2000-01-06       NaN       NaN       NaN
2000-01-07       NaN       NaN       NaN
2000-01-08  0.883390  1.335357  0.579100
2000-01-09  0.498776  0.315503       NaN
2000-01-10  1.232448  2.043798  1.021664


<a id='basics-elementwise'></a>

### Applying elementwise functions

Since not all functions can be vectorized (accept NumPy arrays and return
another array or value), the methods `applymap()` on DataFrame
and analogously `map()` on Series accept any Python function taking
a single value and returning a single value. For example:

In [358]:
df4 = df_orig.copy()
df4

,one,two,three
a,0.075514,1.092052,NaN
b,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
d,NaN,0.295426,-0.720844


In [359]:
def f(x):
    return len(str(x))

df4["one"].map(f)

a    19
b    18
c    19
d     3
Name: one, dtype: int64

In [360]:
df4.applymap(f)

,one,two,three
a,19,17,3
b,18,19,18
c,19,19,18
d,3,19,19


`Series.map()` has an additional feature; it can be used to easily
“link” or “map” values defined by a secondary series. This is closely related
to merging/joining functionality:

In [449]:
s = pd.Series(
    ["six", "seven", "six", "seven", "six"], index=["a", "b", "c", "d", "e"]
)
s


a      six
b    seven
c      six
d    seven
e      six
dtype: object

In [450]:
t = pd.Series({"six": 6.0, "seven": 7.0})
t

six      6.0
seven    7.0
dtype: float64

In [451]:
s.map(t)

a    6.0
b    7.0
c    6.0
d    7.0
e    6.0
dtype: float64


<a id='basics-reindexing'></a>

## Reindexing and altering labels

`reindex()` is the fundamental data alignment method in pandas.
It is used to implement nearly all other features relying on label-alignment
functionality. To *reindex* means to conform the data to match a given set of
labels along a particular axis. This accomplishes several things:

- Reorders the existing data to match a new set of labels  
- Inserts missing value (NA) markers in label locations where no data for
  that label existed  
- If specified, **fill** data for missing labels using logic (highly relevant
  to working with time series data)  


Here is a simple example:

In [362]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
s

In [362]:
s.reindex(["e", "b", "f", "d"])

e   -0.985852
b   -1.011751
f         NaN
d    0.026089
dtype: float64

Here, the `f` label was not contained in the Series and hence appears as
`NaN` in the result.

With a DataFrame, you can simultaneously reindex the index and columns:

In [363]:
df

In [363]:
df.reindex(index=["c", "f", "b"], columns=["three", "two", "one"])

,three,two,one
c,0.334334,0.228052,-1.028668
f,NaN,NaN,NaN
b,0.706416,0.294612,-1.030396


You may also use `reindex` with an `axis` keyword:

In [364]:
df.reindex(["c", "f", "b"], axis="index")

,one,two,three
c,-1.028668,0.228052,0.334334
f,NaN,NaN,NaN
b,-1.030396,0.294612,0.706416


Note that the `Index` objects containing the actual axis labels can be
**shared** between objects. So if we have a Series and a DataFrame, the
following can be done:

In [365]:
rs = s.reindex(df.index)
rs
rs.index is df.index

True

This means that the reindexed Series’s index is the same Python object as the
DataFrame’s index.

`DataFrame.reindex()` also supports an “axis-style” calling convention,
where you specify a single `labels` argument and the `axis` it applies to.

In [366]:
df.reindex(["c", "f", "b"], axis="index")
df.reindex(["three", "two", "one"], axis="columns")

,three,two,one
a,NaN,1.092052,0.075514
b,0.706416,0.294612,-1.030396
c,0.334334,0.228052,-1.028668
d,-0.720844,0.295426,NaN


MultiIndex / Advanced Indexing is an even more concise way of
doing reindexing.

>**Note**
>
>When writing performance-sensitive code, there is a good reason to spend
some time becoming a reindexing ninja: **many operations are faster on
pre-aligned data**. Adding two unaligned DataFrames internally triggers a
reindexing step. For exploratory analysis you will hardly notice the
difference (because `reindex` has been heavily optimized), but when CPU
cycles matter sprinkling a few explicit `reindex` calls here and there can
have an impact.


<a id='basics-reindex-like'></a>

### Reindexing to align with another object

You may wish to take an object and reindex its axes to be labeled the same as
another object. While the syntax for this is straightforward albeit verbose, it
is a common enough operation that the `reindex_like()` method is
available to make this simpler:

In [367]:
df2 = df.reindex(["a", "b", "c"], columns=["one", "two"])
df3 = df2 - df2.mean()

In [368]:
df2

In [368]:
df3

In [368]:
df.reindex_like(df2)

,one,two
a,0.075514,1.092052
b,-1.030396,0.294612
c,-1.028668,0.228052



<a id='basics-align'></a>

### Aligning objects with each other with `align`

The `align()` method is the fastest way to simultaneously align two objects. It
supports a `join` argument (related to joining and merging):

> - `join='outer'`: take the union of the indexes (default)  
- `join='left'`: use the calling object’s index  
- `join='right'`: use the passed object’s index  
- `join='inner'`: intersect the indexes  



It returns a tuple with both of the reindexed Series:

In [369]:
s = pd.Series(np.random.randn(5), index=["a", "b", "c", "d", "e"])
s1 = s[:4]
s1

In [369]:
s2 = s[1:]
s2

In [369]:
s1.align(s2)

In [369]:
s1.align(s2, join="inner")

In [369]:
s1.align(s2, join="left")

(a    1.580015
 b   -0.008817
 c    0.819259
 d   -0.070270
 dtype: float64,
 a         NaN
 b   -0.008817
 c    0.819259
 d   -0.070270
 dtype: float64)


<a id='basics-df-join'></a>
For DataFrames, the join method will be applied to both the index and the
columns by default:

In [370]:
df.align(df2, join="inner")

(        one       two
 a  0.075514  1.092052
 b -1.030396  0.294612
 c -1.028668  0.228052,
         one       two
 a  0.075514  1.092052
 b -1.030396  0.294612
 c -1.028668  0.228052)

You can also pass an `axis` option to only align on the specified axis:

In [371]:
df.align(df2, join="inner", axis=0)

(        one       two     three
 a  0.075514  1.092052       NaN
 b -1.030396  0.294612  0.706416
 c -1.028668  0.228052  0.334334,
         one       two
 a  0.075514  1.092052
 b -1.030396  0.294612
 c -1.028668  0.228052)


<a id='basics-align-frame-series'></a>
If you pass a Series to `DataFrame.align()`, you can choose to align both
objects either on the DataFrame’s index or columns using the `axis` argument:

In [372]:
df.align(df2.iloc[0], axis=1)

(        one     three       two
 a  0.075514       NaN  1.092052
 b -1.030396  0.706416  0.294612
 c -1.028668  0.334334  0.228052
 d       NaN -0.720844  0.295426,
 one      0.075514
 three         NaN
 two      1.092052
 Name: a, dtype: float64)


<a id='basics-reindex-fill'></a>

### Filling while reindexing

`reindex()` takes an optional parameter `method` which is a
filling method chosen from the following table:

|Method|Action|
|:----------------------------|:------------------------------------------------|
|pad / ffill|Fill values forward|
|bfill / backfill|Fill values backward|
|nearest|Fill from the nearest index value|

We illustrate these fill methods on a simple Series:

In [452]:
rng = pd.date_range("1/3/2000", periods=8)
ts = pd.Series(np.random.randn(8), index=rng)
ts2 = ts[[0, 3, 6]]

In [453]:
ts

2000-01-03    1.168028
2000-01-04    0.144824
2000-01-05    0.985497
2000-01-06    0.479851
2000-01-07    0.635126
2000-01-08    0.490108
2000-01-09   -0.008810
2000-01-10    0.848273
Freq: D, dtype: float64

In [454]:
ts2

2000-01-03    1.168028
2000-01-06    0.479851
2000-01-09   -0.008810
Freq: 3D, dtype: float64

In [455]:
ts2.reindex(ts.index)

2000-01-03    1.168028
2000-01-04         NaN
2000-01-05         NaN
2000-01-06    0.479851
2000-01-07         NaN
2000-01-08         NaN
2000-01-09   -0.008810
2000-01-10         NaN
Freq: D, dtype: float64

In [456]:
ts2.reindex(ts.index, method="ffill")

2000-01-03    1.168028
2000-01-04    1.168028
2000-01-05    1.168028
2000-01-06    0.479851
2000-01-07    0.479851
2000-01-08    0.479851
2000-01-09   -0.008810
2000-01-10   -0.008810
Freq: D, dtype: float64

In [457]:
ts2.reindex(ts.index, method="bfill")

2000-01-03    1.168028
2000-01-04    0.479851
2000-01-05    0.479851
2000-01-06    0.479851
2000-01-07   -0.008810
2000-01-08   -0.008810
2000-01-09   -0.008810
2000-01-10         NaN
Freq: D, dtype: float64

In [458]:
ts2.reindex(ts.index, method="nearest")

2000-01-03    1.168028
2000-01-04    1.168028
2000-01-05    0.479851
2000-01-06    0.479851
2000-01-07    0.479851
2000-01-08   -0.008810
2000-01-09   -0.008810
2000-01-10   -0.008810
Freq: D, dtype: float64

These methods require that the indexes are **ordered** increasing or
decreasing.

Note that the same result could have been achieved using
fillna (except for `method='nearest'`) or
interpolate:

In [374]:
ts2.reindex(ts.index).fillna(method="ffill")

2000-01-03    0.145142
2000-01-04    0.145142
2000-01-05    0.145142
2000-01-06   -1.948141
2000-01-07   -1.948141
2000-01-08   -1.948141
2000-01-09   -1.502283
2000-01-10   -1.502283
Freq: D, dtype: float64

`reindex()` will raise a ValueError if the index is not monotonically
increasing or decreasing. `fillna()` and `interpolate()`
will not perform any checks on the order of the index.


<a id='basics-limits-on-reindex-fill'></a>

### Limits on filling while reindexing

The `limit` and `tolerance` arguments provide additional control over
filling while reindexing. Limit specifies the maximum count of consecutive
matches:

In [375]:
ts2.reindex(ts.index, method="ffill", limit=1)

2000-01-03    0.145142
2000-01-04    0.145142
2000-01-05         NaN
2000-01-06   -1.948141
2000-01-07   -1.948141
2000-01-08         NaN
2000-01-09   -1.502283
2000-01-10   -1.502283
Freq: D, dtype: float64

In contrast, tolerance specifies the maximum distance between the index and
indexer values:

In [376]:
ts2.reindex(ts.index, method="ffill", tolerance="1 day")

2000-01-03    0.145142
2000-01-04    0.145142
2000-01-05         NaN
2000-01-06   -1.948141
2000-01-07   -1.948141
2000-01-08         NaN
2000-01-09   -1.502283
2000-01-10   -1.502283
Freq: D, dtype: float64

Notice that when used on a `DatetimeIndex`, `TimedeltaIndex` or
`PeriodIndex`, `tolerance` will coerced into a `Timedelta` if possible.
This allows you to specify tolerance with appropriate strings.


<a id='basics-drop'></a>

### Dropping labels from an axis

A method closely related to `reindex` is the `drop()` function.
It removes a set of labels from an axis:

In [377]:
df
df.drop(["a", "d"], axis=0)
df.drop(["one"], axis=1)

,two,three
a,1.092052,NaN
b,0.294612,0.706416
c,0.228052,0.334334
d,0.295426,-0.720844


Note that the following also works, but is a bit less obvious / clean:

In [378]:
df.reindex(df.index.difference(["a", "d"]))

,one,two,three
b,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334



<a id='basics-rename'></a>

### Renaming / mapping labels

The `rename()` method allows you to relabel an axis based on some
mapping (a dict or Series) or an arbitrary function.

In [379]:
s

In [379]:
s.rename(str.upper)

A    1.580015
B   -0.008817
C    0.819259
D   -0.070270
E    1.272000
dtype: float64

If you pass a function, it must return a value when called with any of the
labels (and must produce a set of unique values). A dict or
Series can also be used:

In [380]:
df.rename(
    columns={"one": "foo", "two": "bar"},
    index={"a": "apple", "b": "banana", "d": "durian"},
)

,foo,bar,three
apple,0.075514,1.092052,NaN
banana,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
durian,NaN,0.295426,-0.720844


If the mapping doesn’t include a column/index label, it isn’t renamed. Note that
extra labels in the mapping don’t throw an error.

`DataFrame.rename()` also supports an “axis-style” calling convention, where
you specify a single `mapper` and the `axis` to apply that mapping to.

In [381]:
df.rename({"one": "foo", "two": "bar"}, axis="columns")
df.rename({"a": "apple", "b": "banana", "d": "durian"}, axis="index")

,one,two,three
apple,0.075514,1.092052,NaN
banana,-1.030396,0.294612,0.706416
c,-1.028668,0.228052,0.334334
durian,NaN,0.295426,-0.720844


The `rename()` method also provides an `inplace` named
parameter that is by default `False` and copies the underlying data. Pass
`inplace=True` to rename the data in place.

Finally, `rename()` also accepts a scalar or list-like
for altering the `Series.name` attribute.

In [382]:
s.rename("scalar-name")

a    1.580015
b   -0.008817
c    0.819259
d   -0.070270
e    1.272000
Name: scalar-name, dtype: float64


<a id='basics-rename-axis'></a>
The methods `DataFrame.rename_axis()` and `Series.rename_axis()`
allow specific names of a `MultiIndex` to be changed (as opposed to the
labels).

In [383]:
df = pd.DataFrame(
    {"x": [1, 2, 3, 4, 5, 6], "y": [10, 20, 30, 40, 50, 60]},
    index=pd.MultiIndex.from_product(
        [["a", "b", "c"], [1, 2]], names=["let", "num"]
    ),
)
df

In [383]:
df.rename_axis(index={"let": "abc"})

In [383]:
df.rename_axis(index=str.upper)

x   y
LET NUM       
a   1    1  10
    2    2  20
b   1    3  30
    2    4  40
c   1    5  50
    2    6  60


<a id='basics-iteration'></a>

## Iteration

The behavior of basic iteration over pandas objects depends on the type.
When iterating over a Series, it is regarded as array-like, and basic iteration
produces the values. DataFrames follow the dict-like convention of iterating
over the “keys” of the objects.

In short, basic iteration (`for i in object`) produces:

- **Series**: values  
- **DataFrame**: column labels  


Thus, for example, iterating over a DataFrame gives you the column names:

In [384]:
df = pd.DataFrame(
    {"col1": np.random.randn(3), "col2": np.random.randn(3)}, index=["a", "b", "c"]
)

for col in df:
    print(col)

col1
col2


pandas objects also have the dict-like `items()` method to
iterate over the (key, value) pairs.

To iterate over the rows of a DataFrame, you can use the following methods:

- `iterrows()`: Iterate over the rows of a DataFrame as (index, Series) pairs.
  This converts the rows to Series objects, which can change the dtypes and has some
  performance implications.  
- `itertuples()`: Iterate over the rows of a DataFrame
  as namedtuples of the values.  This is a lot faster than
  `iterrows()`, and is in most cases preferable to use
  to iterate over the values of a DataFrame.  


Iterating through pandas objects is generally **slow**. In many cases,
iterating manually over the rows is not needed and can be avoided with
one of the following approaches:

- Look for a *vectorized* solution: many operations can be performed using
  built-in methods or NumPy functions, (boolean) indexing, …  
- When you have a function that cannot work on the full DataFrame/Series
  at once, it is better to use `apply()` instead of iterating
  over the values. See the docs on function application.  
- If you need to do iterative manipulations on the values but performance is
  important, consider writing the inner loop with cython or numba.
  See the enhancing performance section for some
  examples of this approach.  


You should **never modify** something you are iterating over.
This is not guaranteed to work in all cases. Depending on the
data types, the iterator returns a copy and not a view, and writing
to it will have no effect!

For example, in the following case setting the value has no effect:

In [385]:
df = pd.DataFrame({"a": [1, 2, 3], "b": ["a", "b", "c"]})

for index, row in df.iterrows():
    row["a"] = 10

df

,a,b
0,1,a
1,2,b
2,3,c


### items

Consistent with the dict-like interface, `items()` iterates
through key-value pairs:

- **Series**: (index, scalar value) pairs  
- **DataFrame**: (column, Series) pairs  


For example:

In [386]:
for label, ser in df.items():
    print(label)
    print(ser)

a
0    1
1    2
2    3
Name: a, dtype: int64
b
0    a
1    b
2    c
Name: b, dtype: object



<a id='basics-iterrows'></a>

### iterrows

`iterrows()` allows you to iterate through the rows of a
DataFrame as Series objects. It returns an iterator yielding each
index value along with a Series containing the data in each row:

In [387]:
for row_index, row in df.iterrows():
    print(row_index, row, sep="\n")

0
a    1
b    a
Name: 0, dtype: object
1
a    2
b    b
Name: 1, dtype: object
2
a    3
b    c
Name: 2, dtype: object


>**Note**
>
>Because `iterrows()` returns a Series for each row,
it does **not** preserve dtypes across the rows (dtypes are
preserved across columns for DataFrames). For example,

In [388]:
df_orig = pd.DataFrame([[1, 1.5]], columns=["int", "float"])
df_orig.dtypes

In [388]:
row = next(df_orig.iterrows())[1]
row

int      1.0
float    1.5
Name: 0, dtype: float64

All values in `row`, returned as a Series, are now upcasted
to floats, also the original integer value in column `x`:

In [389]:
row["int"].dtype

In [389]:
df_orig["int"].dtype

dtype('int64')

To preserve dtypes while iterating over the rows, it is better
to use `itertuples()` which returns namedtuples of the values
and which is generally much faster than `iterrows()`.

For instance, a contrived way to transpose the DataFrame would be:

In [459]:
df2 = pd.DataFrame({"x": [1, 2, 3], "y": [4, 5, 6]})
print(df2)
print('-'*10) 
print(df2.T)
print('-'*10) 
df2_t = pd.DataFrame({idx: values for idx, values in df2.iterrows()})
print(df2_t)

   x  y
0  1  4
1  2  5
2  3  6
----------
   0  1  2
x  1  2  3
y  4  5  6
----------
   0  1  2
x  1  2  3
y  4  5  6


### itertuples

The `itertuples()` method will return an iterator
yielding a namedtuple for each row in the DataFrame. The first element
of the tuple will be the row’s corresponding index value, while the
remaining values are the row values.

For instance:

In [391]:
for row in df.itertuples():
    print(row)

Pandas(Index=0, a=1, b='a')
Pandas(Index=1, a=2, b='b')
Pandas(Index=2, a=3, b='c')


This method does not convert the row to a Series object; it merely
returns the values inside a namedtuple. Therefore,
`itertuples()` preserves the data type of the values
and is generally faster as `iterrows()`.

>**Note**
>
>The column names will be renamed to positional names if they are
invalid Python identifiers, repeated, or start with an underscore.
With a large number of columns (>255), regular tuples are returned.


<a id='basics-dt-accessors'></a>

## .dt accessor

`Series` has an accessor to succinctly return datetime like properties for the
*values* of the Series, if it is a datetime/period like Series.
This will return a Series, indexed like the existing Series.

In [392]:
# datetime
s = pd.Series(pd.date_range("20130101 09:10:12", periods=4))
s

In [392]:
s.dt.hour

In [392]:
s.dt.second

In [392]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

This enables nice expressions like this:

In [393]:
s[s.dt.day == 2]

1   2013-01-02 09:10:12
dtype: datetime64[ns]

You can easily produces tz aware transformations:

In [394]:
stz = s.dt.tz_localize("US/Eastern")
stz

In [394]:
stz.dt.tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

You can also chain these types of operations:

In [395]:
s.dt.tz_localize("UTC").dt.tz_convert("US/Eastern")

0   2013-01-01 04:10:12-05:00
1   2013-01-02 04:10:12-05:00
2   2013-01-03 04:10:12-05:00
3   2013-01-04 04:10:12-05:00
dtype: datetime64[ns, US/Eastern]

You can also format datetime values as strings with `Series.dt.strftime()` which
supports the same format as the standard `strftime()`.

In [396]:
# DatetimeIndex
s = pd.Series(pd.date_range("20130101", periods=4))
s

In [396]:
s.dt.strftime("%Y/%m/%d")

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

In [397]:
# PeriodIndex
s = pd.Series(pd.period_range("20130101", periods=4))
s

In [397]:
s.dt.strftime("%Y/%m/%d")

0    2013/01/01
1    2013/01/02
2    2013/01/03
3    2013/01/04
dtype: object

The `.dt` accessor works for period and timedelta dtypes.

In [398]:
# period
s = pd.Series(pd.period_range("20130101", periods=4, freq="D"))
s

In [398]:
s.dt.year

In [398]:
s.dt.day

0    1
1    2
2    3
3    4
dtype: int64

In [399]:
# timedelta
s = pd.Series(pd.timedelta_range("1 day 00:00:05", periods=4, freq="s"))

,days,hours,minutes,seconds,milliseconds,microseconds,nanoseconds
0,1,0,0,5,0,0,0
1,1,0,0,6,0,0,0
2,1,0,0,7,0,0,0
3,1,0,0,8,0,0,0


In [ ]:
s

In [ ]:
s.dt.days

In [ ]:
s.dt.seconds

In [ ]:
s.dt.components

>**Note**
>
>`Series.dt` will raise a `TypeError` if you access with a non-datetime-like values.

## Vectorized string methods

Series is equipped with a set of string processing methods that make it easy to
operate on each element of the array. Perhaps most importantly, these methods
exclude missing/NA values automatically. These are accessed via the Series’s
`str` attribute and generally have names matching the equivalent (scalar)
built-in string methods. For example:

> 

In [400]:
s = pd.Series(
    ["A", "B", "C", "Aaba", "Baca", np.nan, "CABA", "dog", "cat"], dtype="string"
)
s.str.lower()

0       a
1       b
2       c
3    aaba
4    baca
5    <NA>
6    caba
7     dog
8     cat
dtype: string


Powerful pattern-matching methods are provided as well, but note that
pattern-matching generally uses [regular expressions](https://docs.python.org/3/library/re.html) by default (and in some cases
always uses them).

>**Note**
>
>Prior to pandas 1.0, string methods were only available on `object` -dtype
`Series`. pandas 1.0 added the `StringDtype` which is dedicated
to strings. See text.types for more.

Please see Vectorized String Methods for a complete
description.


<a id='basics-sorting'></a>

## Sorting

pandas supports three kinds of sorting: sorting by index labels,
sorting by column values, and sorting by a combination of both.


<a id='basics-sort-index'></a>

### By index

The `Series.sort_index()` and `DataFrame.sort_index()` methods are
used to sort a pandas object by its index levels.

In [401]:
df = pd.DataFrame(
    {
        "one": pd.Series(np.random.randn(3), index=["a", "b", "c"]),
        "two": pd.Series(np.random.randn(4), index=["a", "b", "c", "d"]),
        "three": pd.Series(np.random.randn(3), index=["b", "c", "d"]),
    }
)

unsorted_df = df.reindex(
    index=["a", "d", "c", "b"], columns=["three", "two", "one"]
)
unsorted_df

In [401]:
# DataFrame
unsorted_df.sort_index()

In [401]:
unsorted_df.sort_index(ascending=False)

In [401]:
unsorted_df.sort_index(axis=1)

In [401]:
# Series
unsorted_df["three"].sort_index()

a         NaN
b    0.874388
c    0.068766
d    1.287203
Name: three, dtype: float64


<a id='basics-sort-index-key'></a>
New in version 1.1.0.

Sorting by index also supports a `key` parameter that takes a callable
function to apply to the index being sorted. For `MultiIndex` objects,
the key is applied per-level to the levels specified by `level`.

In [402]:
s1 = pd.DataFrame({"a": ["B", "a", "C"], "b": [1, 2, 3], "c": [2, 3, 4]}).set_index(
    list("ab")
)
s1

,,c
a,b,
B,1,2
a,2,3
C,3,4


In [403]:
s1.sort_index(level="a")

In [403]:
s1.sort_index(level="a", key=lambda idx: idx.str.lower())

,,c
a,b,
a,2,3
B,1,2
C,3,4


For information on key sorting by value, see value sorting.


<a id='basics-sort-values'></a>

### By values

The `Series.sort_values()` method is used to sort a `Series` by its values. The
`DataFrame.sort_values()` method is used to sort a `DataFrame` by its column or row values.
The optional `by` parameter to `DataFrame.sort_values()` may used to specify one or more columns
to use to determine the sorted order.

In [404]:
df1 = pd.DataFrame(
    {"one": [2, 1, 1, 1], "two": [1, 3, 2, 4], "three": [5, 4, 3, 2]}
)
df1.sort_values(by="two")

,one,two,three
0,2,1,5
2,1,2,3
1,1,3,4
3,1,4,2


The `by` parameter can take a list of column names, e.g.:

In [405]:
df1[["one", "two", "three"]].sort_values(by=["one", "two"])

,one,two,three
2,1,2,3
1,1,3,4
3,1,4,2
0,2,1,5


These methods have special treatment of NA values via the `na_position`
argument:

In [406]:
s[2] = np.nan
s.sort_values()

In [406]:
s.sort_values(na_position="first")

2    <NA>
5    <NA>
0       A
3    Aaba
1       B
4    Baca
6    CABA
8     cat
7     dog
dtype: string


<a id='basics-sort-value-key'></a>
New in version 1.1.0.

Sorting also supports a `key` parameter that takes a callable function
to apply to the values being sorted.

In [407]:
s1 = pd.Series(["B", "a", "C"])

In [408]:
s1.sort_values()

In [408]:
s1.sort_values(key=lambda x: x.str.lower())

1    a
0    B
2    C
dtype: object

`key` will be given the `Series` of values and should return a `Series`
or array of the same shape with the transformed values. For `DataFrame` objects,
the key is applied per column, so the key should still expect a Series and return
a Series, e.g.

In [409]:
df = pd.DataFrame({"a": ["B", "a", "C"], "b": [1, 2, 3]})

In [410]:
df.sort_values(by="a")

In [410]:
df.sort_values(by="a", key=lambda col: col.str.lower())

,a,b
1,a,2
0,B,1
2,C,3


The name or type of each column can be used to apply different functions to
different columns.


<a id='basics-sort-indexes-and-values'></a>

### By indexes and values

Strings passed as the `by` parameter to `DataFrame.sort_values()` may
refer to either columns or index level names.

In [411]:
# Build MultiIndex
idx = pd.MultiIndex.from_tuples(
    [("a", 1), ("a", 2), ("a", 2), ("b", 2), ("b", 1), ("b", 1)]
)
idx.names = ["first", "second"]

# Build DataFrame
df_multi = pd.DataFrame({"A": np.arange(6, 0, -1)}, index=idx)
df_multi

A
first second   
a     1       6
      2       5
      2       4
b     2       3
      1       2
      1       1

Sort by ‘second’ (index) and ‘A’ (column)

In [412]:
df_multi.sort_values(by=["second", "A"])

A
first second   
b     1       1
      1       2
a     1       6
b     2       3
a     2       4
      2       5

>**Note**
>
>If a string matches both a column name and an index level name then a
warning is issued and the column takes precedence. This will result in an
ambiguity error in a future version.


<a id='basics-searchsorted'></a>

### searchsorted

Series has the `searchsorted()` method, which works similarly to
`numpy.ndarray.searchsorted()`.

In [460]:
ser = pd.Series([1, 2, 3])
ser.searchsorted([0, 3])

array([0, 2], dtype=int64)

In [461]:
ser.searchsorted([0, 4])

array([0, 3], dtype=int64)

In [413]:
ser.searchsorted([1, 3], side="right")

In [413]:
ser.searchsorted([1, 3], side="left")

In [413]:
ser = pd.Series([3, 1, 2])

In [413]:
ser.searchsorted([0, 3], sorter=np.argsort(ser))

array([0, 2], dtype=int64)


<a id='basics-nsorted'></a>

### smallest / largest values

`Series` has the `nsmallest()` and `nlargest()` methods which return the
smallest or largest $ n $ values. For a large `Series` this can be much
faster than sorting the entire Series and calling `head(n)` on the result.

In [414]:
s = pd.Series(np.random.permutation(10))
s

In [414]:
s.sort_values()

In [414]:
s.nsmallest(3)

In [414]:
s.nlargest(3)

2    9
7    8
9    7
dtype: int32

`DataFrame` also has the `nlargest` and `nsmallest` methods.

In [415]:
df = pd.DataFrame(
    {
        "a": [-2, -1, 1, 10, 8, 11, -1],
        "b": list("abdceff"),
        "c": [1.0, 2.0, 4.0, 3.2, np.nan, 3.0, 4.0],
    }
)
df.nlargest(3, "a")

In [415]:
df.nlargest(5, ["a", "c"])

In [415]:
df.nsmallest(3, "a")

In [415]:
df.nsmallest(5, ["a", "c"])

,a,b,c
0,-2,a,1.0
1,-1,b,2.0
6,-1,f,4.0
2,1,d,4.0
4,8,e,NaN



<a id='basics-multiindex-sorting'></a>

### Sorting by a MultiIndex column

You must be explicit about sorting when the column is a MultiIndex, and fully specify
all levels to `by`.

In [416]:
df1.columns = pd.MultiIndex.from_tuples(
    [("a", "one"), ("a", "two"), ("b", "three")]
)
df1.sort_values(by=("a", "two"))

a         b
  one two three
0   2   1     5
2   1   2     3
1   1   3     4
3   1   4     2

## Copying

The `copy()` method on pandas objects copies the underlying data (though not
the axis indexes, since they are immutable) and returns a new object. Note that
**it is seldom necessary to copy objects**. For example, there are only a
handful of ways to alter a DataFrame *in-place*:

- Inserting, deleting, or modifying a column.  
- Assigning to the `index` or `columns` attributes.  
- For homogeneous data, directly modifying the values via the `values`
  attribute or advanced indexing.  


To be clear, no pandas method has the side effect of modifying your data;
almost every method returns a new object, leaving the original object
untouched. If the data is modified, it is because you did so explicitly.


<a id='basics-dtypes'></a>

## dtypes

For the most part, pandas uses NumPy arrays and dtypes for Series or individual
columns of a DataFrame. NumPy provides support for `float`,
`int`, `bool`, `timedelta64[ns]` and `datetime64[ns]` (note that NumPy
does not support timezone-aware datetimes).

pandas and third-party libraries *extend* NumPy’s type system in a few places.
This section describes the extensions pandas has made internally.
See extending.extension-types for how to write your own extension that
works with pandas. See [Extension data types](../ecosystem.ipynb#ecosystem-extensions) for a list of third-party
libraries that have implemented an extension.

The following table lists all of pandas extension types. For methods requiring `dtype`
arguments, strings can be specified as indicated. See the respective
documentation sections for more on each type.


| Kind of Data        | Data Type                                                    | Scalar                                                       | Array                                                        | String Aliases                                | Documentation                                                |
| :------------------ | :----------------------------------------------------------- | :----------------------------------------------------------- | :----------------------------------------------------------- | :----------------------------------------------------------- | :----------------------------------------------------------- |
| tz-aware datetime   | [`DatetimeTZDtype`](https://pandas.pydata.org/docs/reference/api/pandas.DatetimeTZDtype.html#pandas.DatetimeTZDtype) | [`Timestamp`](https://pandas.pydata.org/docs/reference/api/pandas.Timestamp.html#pandas.Timestamp) | [`arrays.DatetimeArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.DatetimeArray.html#pandas.arrays.DatetimeArray) | `'datetime64[ns, <tz>]'`                                     | [Time zone handling](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-timezone) |
| Categorical         | [`CategoricalDtype`](https://pandas.pydata.org/docs/reference/api/pandas.CategoricalDtype.html#pandas.CategoricalDtype) | (none)                                                       | [`Categorical`](https://pandas.pydata.org/docs/reference/api/pandas.Categorical.html#pandas.Categorical) | `'category'`                                                 | [Categorical data](https://pandas.pydata.org/docs/user_guide/categorical.html#categorical) |
| period (time spans) | [`PeriodDtype`](https://pandas.pydata.org/docs/reference/api/pandas.PeriodDtype.html#pandas.PeriodDtype) | [`Period`](https://pandas.pydata.org/docs/reference/api/pandas.Period.html#pandas.Period) | [`arrays.PeriodArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.PeriodArray.html#pandas.arrays.PeriodArray) | `'period[<freq>]'`, `'Period[<freq>]'`                       | [Time span representation](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-periods) |
| sparse              | [`SparseDtype`](https://pandas.pydata.org/docs/reference/api/pandas.SparseDtype.html#pandas.SparseDtype) | (none)                                                       | [`arrays.SparseArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.SparseArray.html#pandas.arrays.SparseArray) | `'Sparse'`, `'Sparse[int]'`, `'Sparse[float]'`               | [Sparse data structures](https://pandas.pydata.org/docs/user_guide/sparse.html#sparse) |
| intervals           | [`IntervalDtype`](https://pandas.pydata.org/docs/reference/api/pandas.IntervalDtype.html#pandas.IntervalDtype) | [`Interval`](https://pandas.pydata.org/docs/reference/api/pandas.Interval.html#pandas.Interval) | [`arrays.IntervalArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.IntervalArray.html#pandas.arrays.IntervalArray) | `'interval'`, `'Interval'`, `'Interval[<numpy_dtype>]'`, `'Interval[datetime64[ns, <tz>]]'`, `'Interval[timedelta64[<freq>]]'` | [IntervalIndex](https://pandas.pydata.org/docs/user_guide/advanced.html#advanced-intervalindex) |
| nullable integer    | [`Int64Dtype`](https://pandas.pydata.org/docs/reference/api/pandas.Int64Dtype.html#pandas.Int64Dtype), … | (none)                                                       | [`arrays.IntegerArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.IntegerArray.html#pandas.arrays.IntegerArray) | `'Int8'`, `'Int16'`, `'Int32'`, `'Int64'`, `'UInt8'`, `'UInt16'`, `'UInt32'`, `'UInt64'` | [Nullable integer data type](https://pandas.pydata.org/docs/user_guide/integer_na.html#integer-na) |
|                     |                                                              |                                                              |                                                              |                                                              |                                                              |
| Strings             | [`StringDtype`](https://pandas.pydata.org/docs/reference/api/pandas.StringDtype.html#pandas.StringDtype) | [`str`](https://docs.python.org/3/library/stdtypes.html#str) | [`arrays.StringArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.StringArray.html#pandas.arrays.StringArray) | `'string'`                                                   | [Working with text data](https://pandas.pydata.org/docs/user_guide/text.html#text) |
| Boolean (with NA)   | [`BooleanDtype`](https://pandas.pydata.org/docs/reference/api/pandas.BooleanDtype.html#pandas.BooleanDtype) | [`bool`](https://docs.python.org/3/library/functions.html#bool) | [`arrays.BooleanArray`](https://pandas.pydata.org/docs/reference/api/pandas.arrays.BooleanArray.html#pandas.arrays.BooleanArray) | `'boolean'`                                                  | [Boolean data with missing values](https://pandas.pydata.org/docs/reference/arrays.html#api-arrays-bool) |

pandas has two ways to store strings.

1. `object` dtype, which can hold any Python object, including strings.  
1. `StringDtype`, which is dedicated to strings.  


Generally, we recommend using `StringDtype`. See text.types for more.

Finally, arbitrary objects may be stored using the `object` dtype, but should
be avoided to the extent possible (for performance and interoperability with
other libraries and methods. See basics.object_conversion).

A convenient `dtypes` attribute for DataFrame returns a Series
with the data type of each column.

In [417]:
dft = pd.DataFrame(
    {
        "A": np.random.rand(3),
        "B": 1,
        "C": "foo",
        "D": pd.Timestamp("20010102"),
        "E": pd.Series([1.0] * 3).astype("float32"),
        "F": False,
        "G": pd.Series([1] * 3, dtype="int8"),
    }
)
dft

In [417]:
dft.dtypes

A           float64
B             int64
C            object
D    datetime64[ns]
E           float32
F              bool
G              int8
dtype: object

On a `Series` object, use the `dtype` attribute.

In [418]:
dft["A"].dtype

dtype('float64')

If a pandas object contains data with multiple dtypes *in a single column*, the
dtype of the column will be chosen to accommodate all of the data types
(`object` is the most general).

In [419]:
# these ints are coerced to floats
pd.Series([1, 2, 3, 4, 5, 6.0])

In [419]:
# string data forces an ``object`` dtype
pd.Series([1, 2, 3, 6.0, "foo"])

0      1
1      2
2      3
3    6.0
4    foo
dtype: object

The number of columns of each type in a `DataFrame` can be found by calling
`DataFrame.dtypes.value_counts()`.

In [420]:
dft.dtypes.value_counts()

int64             1
bool              1
float32           1
float64           1
int8              1
datetime64[ns]    1
object            1
dtype: int64

Numeric dtypes will propagate and can coexist in DataFrames.
If a dtype is passed (either directly via the `dtype` keyword, a passed `ndarray`,
or a passed `Series`), then it will be preserved in DataFrame operations. Furthermore,
different numeric dtypes will **NOT** be combined. The following example will give you a taste.

In [421]:
df1 = pd.DataFrame(np.random.randn(8, 1), columns=["A"], dtype="float32")
df1

In [421]:
df1.dtypes

In [421]:
df2 = pd.DataFrame(
    {
        "A": pd.Series(np.random.randn(8), dtype="float16"),
        "B": pd.Series(np.random.randn(8)),
        "C": pd.Series(np.array(np.random.randn(8), dtype="uint8")),
    }
)
df2

In [421]:
df2.dtypes

A    float16
B    float64
C      uint8
dtype: object

### defaults

By default integer types are `int64` and float types are `float64`,
*regardless* of platform (32-bit or 64-bit).
The following will all result in `int64` dtypes.

In [422]:
pd.DataFrame([1, 2], columns=["a"]).dtypes

In [422]:
pd.DataFrame({"a": [1, 2]}).dtypes

In [422]:
pd.DataFrame({"a": 1}, index=list(range(2))).dtypes

a    int64
dtype: object

Note that Numpy will choose *platform-dependent* types when creating arrays.
The following **WILL** result in `int32` on 32-bit platform.

In [423]:
frame = pd.DataFrame(np.array([1, 2]))

### upcasting

Types can potentially be *upcasted* when combined with other types, meaning they are promoted
from the current type (e.g. `int` to `float`).

In [424]:
df3 = df1.reindex_like(df2).fillna(value=0.0) + df2
df3

In [424]:
df3.dtypes

A    float32
B    float64
C    float64
dtype: object

`DataFrame.to_numpy()` will return the *lower-common-denominator* of the dtypes, meaning
the dtype that can accommodate **ALL** of the types in the resulting homogeneous dtyped NumPy array. This can
force some *upcasting*.

In [425]:
df3.to_numpy().dtype

dtype('float64')

### astype


<a id='basics-cast'></a>
You can use the `astype()` method to explicitly convert dtypes from one to another. These will by default return a copy,
even if the dtype was unchanged (pass `copy=False` to change this behavior). In addition, they will raise an
exception if the astype operation is invalid.

Upcasting is always according to the **NumPy** rules. If two different dtypes are involved in an operation,
then the more *general* one will be used as the result of the operation.

In [426]:
df3

In [426]:
df3.dtypes

In [426]:
# conversion of dtypes
df3.astype("float32").dtypes

A    float32
B    float32
C    float32
dtype: object

Convert a subset of columns to a specified type using `astype()`.

In [427]:
dft = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6], "c": [7, 8, 9]})
dft[["a", "b"]] = dft[["a", "b"]].astype(np.uint8)
dft

In [427]:
dft.dtypes

a    uint8
b    uint8
c    int64
dtype: object

Convert certain columns to a specific dtype by passing a dict to `astype()`.

In [428]:
dft1 = pd.DataFrame({"a": [1, 0, 1], "b": [4, 5, 6], "c": [7, 8, 9]})
dft1 = dft1.astype({"a": np.bool_, "c": np.float64})
dft1

In [428]:
dft1.dtypes

a       bool
b      int64
c    float64
dtype: object

>**Note**
>
>When trying to convert a subset of columns to a specified type using `astype()`  and `loc()`, upcasting occurs.

`loc()` tries to fit in what we are assigning to the current dtypes, while `[]` will overwrite them taking the dtype from the right hand side. Therefore the following piece of code produces the unintended result.

In [429]:
dft = pd.DataFrame({"a": [1, 2, 3], "b": [4, 5, 6], "c": [7, 8, 9]})
dft.loc[:, ["a", "b"]].astype(np.uint8).dtypes

In [429]:
dft.loc[:, ["a", "b"]] = dft.loc[:, ["a", "b"]].astype(np.uint8)

In [429]:
dft.dtypes

a    int64
b    int64
c    int64
dtype: object


<a id='basics-object-conversion'></a>

### object conversion

pandas offers various functions to try to force conversion of types from the `object` dtype to other types.
In cases where the data is already of the correct type, but stored in an `object` array, the
`DataFrame.infer_objects()` and `Series.infer_objects()` methods can be used to soft convert
to the correct type.

> 

In [430]:
import datetime

df = pd.DataFrame(
    [
        [1, 2],
        ["a", "b"],
        [datetime.datetime(2016, 3, 2), datetime.datetime(2016, 3, 2)],
    ]
)
df = df.T
df

In [430]:
df.dtypes

0            object
1            object
2    datetime64[ns]
dtype: object


Because the data was transposed the original inference stored all columns as object, which
`infer_objects` will correct.

> 

In [431]:
df.infer_objects().dtypes

0             int64
1            object
2    datetime64[ns]
dtype: object


The following functions are available for one dimensional object arrays or scalars to perform
hard conversion of objects to a specified type:

- `to_numeric()` (conversion to numeric dtypes)  

In [432]:
m = ["1.1", 2, 3]
pd.to_numeric(m)

array([1.1, 2. , 3. ])

- `to_datetime()` (conversion to datetime objects)  

In [433]:
import datetime

m = ["2016-07-09", datetime.datetime(2016, 3, 2)]
pd.to_datetime(m)

DatetimeIndex(['2016-07-09', '2016-03-02'], dtype='datetime64[ns]', freq=None)

- `to_timedelta()` (conversion to timedelta objects)  

In [434]:
m = ["5us", pd.Timedelta("1day")]
pd.to_timedelta(m)

TimedeltaIndex(['0 days 00:00:00.000005', '1 days 00:00:00'], dtype='timedelta64[ns]', freq=None)



To force a conversion, we can pass in an `errors` argument, which specifies how pandas should deal with elements
that cannot be converted to desired dtype or object. By default, `errors='raise'`, meaning that any errors encountered
will be raised during the conversion process. However, if `errors='coerce'`, these errors will be ignored and pandas
will convert problematic elements to `pd.NaT` (for datetime and timedelta) or `np.nan` (for numeric). This might be
useful if you are reading in data which is mostly of the desired dtype (e.g. numeric, datetime), but occasionally has
non-conforming elements intermixed that you want to represent as missing:

In [435]:
import datetime

m = ["apple", datetime.datetime(2016, 3, 2)]
pd.to_datetime(m, errors="coerce")

In [435]:
m = ["apple", 2, 3]
pd.to_numeric(m, errors="coerce")

In [435]:
m = ["apple", pd.Timedelta("1day")]
pd.to_timedelta(m, errors="coerce")

TimedeltaIndex([NaT, '1 days'], dtype='timedelta64[ns]', freq=None)

The `errors` parameter has a third option of `errors='ignore'`, which will simply return the passed in data if it
encounters any errors with the conversion to a desired data type:

In [436]:
import datetime

m = ["apple", datetime.datetime(2016, 3, 2)]
pd.to_datetime(m, errors="ignore")

In [436]:
m = ["apple", 2, 3]
pd.to_numeric(m, errors="ignore")

In [436]:
m = ["apple", pd.Timedelta("1day")]
pd.to_timedelta(m, errors="ignore")

array(['apple', Timedelta('1 days 00:00:00')], dtype=object)

In addition to object conversion, `to_numeric()` provides another argument `downcast`, which gives the
option of downcasting the newly (or already) numeric data to a smaller dtype, which can conserve memory:

In [480]:
m = ["1", 2, 32768]
pd.to_numeric(m, downcast="integer")  # smallest signed int dtype

array([    1,     2, 32768], dtype=int32)

In [479]:
m = ["1", 2, 32767]
pd.to_numeric(m, downcast="signed")  # same as 'integer'

array([    1,     2, 32767], dtype=int16)

In [485]:
m = ["1", 2, 65535]
pd.to_numeric(m, downcast="unsigned")  # smallest unsigned int dtype

array([    1,     2, 65535], dtype=uint16)

In [486]:
pd.to_numeric(m, downcast="float")  # smallest float dtype

array([1.0000e+00, 2.0000e+00, 6.5535e+04], dtype=float32)

As these methods apply only to one-dimensional arrays, lists or scalars; they cannot be used directly on multi-dimensional objects such
as DataFrames. However, with `apply()`, we can “apply” the function over each column efficiently:

In [487]:
import datetime
df = pd.DataFrame([["2016-07-09", datetime.datetime(2016, 3, 2)]] * 2, dtype="O")
df

,0,1
0,2016-07-09,2016-03-02 00:00:00
1,2016-07-09,2016-03-02 00:00:00


In [488]:
df.apply(pd.to_datetime)

,0,1
0,2016-07-09,2016-03-02
1,2016-07-09,2016-03-02


In [489]:
df = pd.DataFrame([["1.1", 2, 3]] * 2, dtype="O")
df

,0,1,2
0,1.1,2,3
1,1.1,2,3


In [490]:
df.apply(pd.to_numeric)

,0,1,2
0,1.1,2,3
1,1.1,2,3


In [491]:
df = pd.DataFrame([["5us", pd.Timedelta("1day")]] * 2, dtype="O")
df

,0,1
0,5us,1 days 00:00:00
1,5us,1 days 00:00:00


In [492]:
df.apply(pd.to_timedelta)

,0,1
0,0 days 00:00:00.000005,1 days
1,0 days 00:00:00.000005,1 days


### gotchas

Performing selection operations on `integer` type data can easily upcast the data to `floating`.
The dtype of the input data will be preserved in cases where `nans` are not introduced.
See also [Support for integer NA](43_FAQ_gotchas.ipynb#gotchas-intna).

In [439]:
dfi = df3.astype("int32")
dfi["E"] = 1
dfi

In [439]:
dfi.dtypes

In [439]:
casted = dfi[dfi > 0]

In [439]:
casted

In [439]:
casted.dtypes

A    float64
B    float64
C    float64
E      int64
dtype: object

While float dtypes are unchanged.

In [440]:
dfa = df3.copy()
dfa["A"] = dfa["A"].astype("float32")
dfa.dtypes

In [440]:
casted = dfa[df2 > 0]
casted

In [440]:
casted.dtypes

A    float32
B    float64
C    float64
dtype: object

## Selecting columns based on `dtype`


<a id='basics-selectdtypes'></a>
The `select_dtypes()` method implements subsetting of columns
based on their `dtype`.

First, let’s create a `DataFrame` with a slew of different
dtypes:

In [493]:
df = pd.DataFrame(
    {
        "string": list("abc"),
        "int64": list(range(1, 4)),
        "uint8": np.arange(3, 6).astype("u1"),
        "float64": np.arange(4.0, 7.0),
        "bool1": [True, False, True],
        "bool2": [False, True, False],
        "dates": pd.date_range("now", periods=3),
        "category": pd.Series(list("ABC")).astype("category"),
    }
)
df["tdeltas"] = df.dates.diff()
df["uint64"] = np.arange(3, 6).astype("u8")
df["other_dates"] = pd.date_range("20130101", periods=3)
df["tz_aware_dates"] = pd.date_range("20130101", periods=3, tz="US/Eastern")
df

,string,int64,uint8,float64,bool1,bool2,dates,category,tdeltas,uint64,other_dates,tz_aware_dates
0,a,1,3,4.0,True,False,2021-08-30 09:55:01.784984,A,NaT,3,2013-01-01,2013-01-01 00:00:00-05:00
1,b,2,4,5.0,False,True,2021-08-31 09:55:01.784984,B,1 days,4,2013-01-02,2013-01-02 00:00:00-05:00
2,c,3,5,6.0,True,False,2021-09-01 09:55:01.784984,C,1 days,5,2013-01-03,2013-01-03 00:00:00-05:00


And the dtypes:

In [442]:
df.dtypes

string                                object
int64                                  int64
uint8                                  uint8
float64                              float64
bool1                                   bool
bool2                                   bool
dates                         datetime64[ns]
category                            category
tdeltas                      timedelta64[ns]
uint64                                uint64
other_dates                   datetime64[ns]
tz_aware_dates    datetime64[ns, US/Eastern]
dtype: object

`select_dtypes()` has two parameters `include` and `exclude` that allow you to
say “give me the columns *with* these dtypes” (`include`) and/or “give the
columns *without* these dtypes” (`exclude`).

For example, to select `bool` columns:

In [443]:
df.select_dtypes(include=[bool])

,bool1,bool2
0,True,False
1,False,True
2,True,False


You can also pass the name of a dtype in the [NumPy dtype hierarchy](https://numpy.org/doc/stable/reference/arrays.scalars.html):

In [444]:
df.select_dtypes(include=["bool"])

,bool1,bool2
0,True,False
1,False,True
2,True,False


`select_dtypes()` also works with generic dtypes as well.

For example, to select all numeric and boolean columns while excluding unsigned
integers:

In [445]:
df.select_dtypes(include=["number", "bool"], exclude=["unsignedinteger"])

,int64,float64,bool1,bool2,tdeltas
0,1,4.0,True,False,NaT
1,2,5.0,False,True,1 days
2,3,6.0,True,False,1 days


To select string columns you must use the `object` dtype:

In [446]:
df.select_dtypes(include=["object"])

,string
0,a
1,b
2,c


To see all the child dtypes of a generic `dtype` like `numpy.number` you
can define a function that returns a tree of child dtypes:

In [447]:
def subdtypes(dtype):
    subs = dtype.__subclasses__()
    if not subs:
        return dtype
    return [dtype, [subdtypes(dt) for dt in subs]]

All NumPy dtypes are subclasses of `numpy.generic`:

In [448]:
subdtypes(np.generic)

[numpy.generic,
 [[numpy.number,
   [[numpy.integer,
     [[numpy.signedinteger,
       [numpy.int8,
        numpy.int16,
        numpy.intc,
        numpy.int32,
        numpy.int64,
        numpy.timedelta64]],
      [numpy.unsignedinteger,
       [numpy.uint8, numpy.uint16, numpy.uintc, numpy.uint32, numpy.uint64]]]],
    [numpy.inexact,
     [[numpy.floating,
       [numpy.float16, numpy.float32, numpy.float64, numpy.longdouble]],
      [numpy.complexfloating,
       [numpy.complex64, numpy.complex128, numpy.clongdouble]]]]]],
  [numpy.flexible,
   [[numpy.character, [numpy.bytes_, numpy.str_]],
    [numpy.void, [numpy.record]]]],
  numpy.bool_,
  numpy.datetime64,
  numpy.object_]]

>**Note**
>
>pandas also defines the types `category`, and `datetime64[ns, tz]`, which are not integrated into the normal
NumPy hierarchy and won’t show up with the above function.